# Elasticsearch Vector Store

In [5]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

#### Load documents, build VectorStoreIndex with Elasticsearch

In [6]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ElasticsearchStore
from IPython.display import Markdown, display

In [7]:
# load documents
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

In [ ]:
# initialize without metadata filter
from llama_index.storage.storage_context import StorageContext

vector_store = ElasticsearchStore(
    es_url="http://localhost:9200", index_name="paul_graham"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

#### Query Index

In [15]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author learn?")
print(response)

INFO:elastic_transport.transport:POST http://localhost:9200/paul_graham/_search [status:200 duration:0.030s]
POST http://localhost:9200/paul_graham/_search [status:200 duration:0.030s]
POST http://localhost:9200/paul_graham/_search [status:200 duration:0.030s]
The author learned several things during their time at Interleaf. They learned that it's better for technology companies to be run by product people than sales people, that code edited by too many people leads to bugs, that cheap office space can be depressing, that planned meetings are inferior to corridor conversations, that big bureaucratic customers can be a dangerous source of money, and that there's not much overlap between conventional office hours and the optimal time for hacking.
